In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import matplotlib.dates as mdates
import geopandas as gpd
from mapclassify import Quantiles
import pandas as pd
import numpy as np
from shapely import wkt         
import matplotlib.pyplot as plt
dr = DataReg()

In [ ]:
pr_zips = gpd.GeoDataFrame(dr.make_spatial_table().df())
pr_zips["geometry"] = pr_zips["geometry"].apply(wkt.loads)
pr_zips = pr_zips.set_geometry("geometry")
pr_zips["zipcode"] = pr_zips["zipcode"].astype(str)
pr_zips

In [ ]:
df = pl.read_parquet("data/impact.parquet").with_columns(pl.col("zipcode").str.replace("zip_",""))
gdf = pr_zips.join(
                df.to_pandas().set_index("zipcode"), on="zipcode", how="inner"
        ).reset_index(drop=True)
gdf

In [ ]:
# Assuming gdf is your GeoDataFrame
data = gdf["impact"].round()

# Define custom bins such that 0 is in the center
quantile_values = np.percentile(data[data != 0], [0, 25, 50, 75, 100])

# Make sure 0 is in the middle, adjust bins around it
bins = list(quantile_values)

# Create a custom classification using Quantiles
classifier = Quantiles(data, k=len(bins)-1)

# Manually assign the bins to the GeoDataFrame based on your custom bins
gdf["impact_bins"] = pd.cut(data, bins=bins, include_lowest=True)

# Plot with custom binning and 'viridis' colormap
gdf.plot(
    column="impact_bins",
    legend=True,
    figsize=(15, 10),
    cmap='viridis',  # Apply the 'viridis' colormap
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
        "hatch": "///",
        "label": "Missing values",
    },
)